In [1]:
import numpy as np
import h5py as h5

In [2]:
cd

/home/s1949330


In [3]:
cd Documents/initial_data

/home/s1949330/Documents/initial_data


In [4]:
class readGEDI():
    def __init__(self,fileList):
        gedi_f = h5.File('GEDI02_B_2020_04_30_2020124194427_O07882_01_T02809_02_003_01_V002.h5', 'r')
        beamList = ['BEAM0000','BEAM0001','BEAM0010','BEAM0011','BEAM0101','BEAM0110','BEAM1000','BEAM1011']       
        
        self.x = np.empty((), dtype = float)
        self.y = np.empty((), dtype = float)
        self.sens = np.empty((), dtype = float)
        self.cover = np.empty((), dtype = float)
        
        for beam in beamList:
            self.x = np.append(self.x, np.array(gedi_f[beam]['geolocation/lon_lowestmode']))
            self.y = np.append(self.y, np.array(gedi_f[beam]['geolocation/lat_lowestmode']))
            self.sens = np.append(self.sens, np.array(gedi_f[beam]['sensitivity']))
            self.cover = np.append(self.cover, np.array(gedi_f[beam]['cover']))
        

In [5]:
gedi = readGEDI('fileList')
print(gedi.x)
print(gedi.y)
print(gedi.sens)
print(gedi.cover)

[-0.1        -0.17432582 -0.17403249 ... 43.89469689 43.89499315
 43.8952898 ]
[  0.         -43.36818741 -43.36804818 ...  -0.07988222  -0.07946151
  -0.07904034]
[0.         4.61821699 4.1790204  ... 0.76948482 0.78958929 0.72204185]
[ 2.00000000e+00 -9.99900000e+03 -9.99900000e+03 ...  1.97091792e-02
  5.04121557e-03  1.79270748e-02]


In [6]:
import osr
import gdal

class readSentinel():
    def __init__(self,fileName):
        ds = gdal.Open('T36LWK_A025400_20200503T075551_B01.jp2')
        proj = osr.SpatialReference(wkt = ds.GetProjection())
        self.epsg = int(proj.GetAttrValue('AUTHORITY', 1))
        
        self.nX = ds.RasterXSize
        self.nY = ds.RasterYSize
        transform_ds = ds.GetGeoTransform()
        self.xOrigin = transform_ds[0]
        self.yOrigin = transform_ds[3]
        self.pixelWidth = transform_ds[1]
        self.pixelHeight = transform_ds[5]
        
        self.data = ds.GetRasterBand(1).ReadAsArray(0, 0, self.nX, self.nY)
        
        s2 = np.empty((12), dtype = readSentinel)
        
        i = 0
        for band in fileName:
            s2[i] = readSentinel(band)
            print(s2[i].epsg)
            i = i + 1

In [7]:
# sentinel2 = readSentinel('fileName')
# print(sentinel2.nX)
# print(sentinel2.nY)
# print(sentinel2.data)